In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
os.listdir(os.getcwd())

['.ipynb_checkpoints',
 'clean_data.csv',
 'clean_data.ipynb',
 'Sarcasm-Detection',
 'source',
 'test-balanced.csv',
 'train-balanced.csv']

In [3]:
#read the training data from the csv file
header = ['label','comment']
data = pd.read_table('train-balanced.csv',
                    sep='\t', 
                    delimiter=',', 
                    names=header,
                    usecols=[0,1],
                    dtype={'label':int,'comment':str},
                    keep_default_na=False)

In [4]:
#helper function to clean the comments
def comment_clean(user_comment):
    #remove the # from hashtag
    if '#' in user_comment:
        user_comment = [term.replace('#','') for term in user_comment]
    #remove the URL links from comments 
    if 'http:/' in user_comment:
        # url of the form [link name](http://url)
        url_link = re.search('\[(.*)\(http:/(.*)\)', user_comment)
        if url_link is not None:
            user_comment = user_comment.replace(url_link.group(0),'')
        else:
            #url of the form http:/
            url_link = re.search('http:/(.*)', user_comment)
            if url_link is not None:
                user_comment = user_comment.replace(url_link.group(0),'')                
    return user_comment           

In [5]:
data['comment'] = data.comment.apply(comment_clean)

In [6]:
#write the cleaned data into a csv file
data.to_csv('clean_data.csv',
           sep= '|')